In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
from account_credentials import LOGIN, PASSWORD, SERVER
from datetime import datetime, timedelta

In [2]:
is_initialized = mt5.initialize()
print('initialize: ', is_initialized)

is_logged_in = mt5.login(LOGIN, PASSWORD, SERVER)
print('logged in: ', is_logged_in)
print('\n')
account_info = mt5.account_info()
print(datetime.now(),
    '| Login: ', account_info.login,
    '| Balance: ', account_info.balance,
    '| Equity: ' , account_info.equity)

initialize:  True
logged in:  True


2023-04-16 09:03:18.425761 | Login:  114123121 | Balance:  503.22 | Equity:  503.22


In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta

# connect to MetaTrader 5
mt5.initialize()

# set the time range for the history
start_time = datetime.now() - timedelta(days=100)
end_time = datetime.now()

# retrieve the history
history = mt5.history_deals_get(start_time, end_time)

# Create an empty dataframe
df = pd.DataFrame(columns=['Time','Deal ID', 'Order Ticket', 'Type', 'Volume', 'Price', 'Profit', 'Comment', 'Magic'])

# Iterate through the trade history and create a list of dataframes
dfs = []
for deal in history:
    row = {'Time': pd.Timestamp(deal.time, unit='s'),
           'Deal ID': deal.order,
           'Order Ticket': deal.ticket,
           'Type': 'Buy' if deal.type == 0 else 'Sell',
           'Volume': deal.volume,
           'Price': deal.price,
           'Profit': deal.profit,
           'Comment': deal.comment,
           'Magic': deal.magic}
    dfs.append(pd.DataFrame(row, index=[0]))

# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Print the resulting dataframe
print(df)



                   Time    Deal ID  Order Ticket  Type  Volume     Price   
0   2023-03-23 01:46:47          0     143696145  Sell    0.00     0.000  \
1   2023-03-28 02:09:05  266948417     145348938   Buy    0.01  1960.969   
2   2023-03-28 02:29:00  266953667     145353163   Buy    0.01  1960.427   
3   2023-03-28 03:03:56  266962782     145360604   Buy    0.02  1961.108   
4   2023-03-28 04:18:59  266982644     145377761  Sell    0.02  1958.718   
..                  ...        ...           ...   ...     ...       ...   
194 2023-04-14 17:08:39  276115481     151490220  Sell    0.01  2002.887   
195 2023-04-14 18:00:00  276131838     151502811   Buy    0.01  2004.088   
196 2023-04-14 18:00:00  276131851     151502821   Buy    0.01  2004.088   
197 2023-04-14 19:17:22  276151472     151514401  Sell    0.01  2007.213   
198 2023-04-14 19:17:22  276151471     151514402  Sell    0.01  2007.213   

     Profit                       Comment   Magic  
0    500.00  D-trial-USD-INT-98b286

In [4]:
df_adjust = df[(df['Profit'] != 0) & (df['Time'] > '2023-3-31')]
print(df_adjust)
df_adjust.to_csv('history_price.csv')

                   Time    Deal ID  Order Ticket  Type  Volume     Price   
80  2023-03-31 07:19:03  268670643     146730069  Sell    0.01  1981.207  \
82  2023-03-31 11:29:47  268798170     146835994  Sell    0.01  1981.799   
84  2023-03-31 19:05:11  269020107     147026872  Sell    0.01  1968.435   
86  2023-04-03 01:45:13  269391156     147259605  Sell    0.01  1959.667   
88  2023-04-03 08:22:51  269581073     147388957  Sell    0.01  1965.072   
90  2023-04-03 10:12:41  269629594     147429873  Sell    0.01  1969.259   
92  2023-04-03 11:31:40  269680116     147468589  Sell    0.01  1976.210   
94  2023-04-03 13:31:12  269750718     147525083  Sell    0.01  1975.620   
96  2023-04-03 15:31:01  269884319     147617571  Sell    0.01  1989.307   
98  2023-04-04 14:00:09  270407933     147976264  Sell    0.01  1995.760   
100 2023-04-04 15:30:20  270563300     148092273  Sell    0.01  2019.158   
102 2023-04-04 16:07:56  270587467     148112967  Sell    0.01  2023.461   
104 2023-04-

In [129]:
print("------MODEL--------")
# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123999]))
m1_df = df_adjust[df_adjust["Magic"] == 123999]
Profit = sum(m1_df["Profit"])
print(f"Profit ${Profit:.2f}")
Accuracy = (m1_df["Profit"] > 0).sum() / len(m1_df)
print(f"Accuracy {Accuracy*100:.2f}%")


Lost_standard = ((m1_df["Profit"] < 0).sum() * TPSL_base) #Positive Value
Lost_actual = (sum(m1_df[m1_df["Profit"] < 0]["Profit"])) # Negative Value
Lost_error = Lost_standard + Lost_actual # if result = positive then Lost_standard > Lost_actual
print(f"Lost Error ${Lost_error:.2f}")

Win_standard = ((m1_df["Profit"] > 0).sum() * TPSL_base) #Positive Value
Win_actual = (sum(m1_df[m1_df["Profit"] > 0]["Profit"])) # Positive Value
Win_error = Win_actual - Win_standard # if result = positive then the actual win is higher than standard win
print(f"Win Error ${Win_error:.2f}")



------MODEL--------
All Samples 30
Profit $19.25
Accuracy 60.00%
Lost Error $0.09
Win Error $0.38


In [92]:
print("------MODEL v2--------")
# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123992]))
print()
b2_buy = df_adjust[(df_adjust["Magic"] == 123992) & (df_adjust["Type"] == "Buy")]
b2_buy_profit = b2_buy["Profit"]
Profit_buy = round(sum(b2_buy_profit),2)
print("Buy samples: ",len(b2_buy_profit))
print("Sum profit b2 buy",Profit_buy)
Accuracy_buy = round((b2_buy["Profit"] > 0).sum() / len(b2_buy["Profit"]),2)
print(f"Accuracy {Accuracy_buy*100:.2f}%")
standard_buy = ((b2_buy["Profit"] > 0).sum()*TPSL_base)-((b2_buy["Profit"] < 0).sum()*TPSL_base)
error_buy = Profit_buy - standard_buy
print("Error buy",error_buy)

print()

b2_sell = df_adjust[(df_adjust["Magic"] == 123992) & (df_adjust["Type"] == "Sell")]
b2_sell_profit = b2_sell["Profit"]
Profit_sell = round(sum(b2_sell_profit),2)
print("Sell samples: ",len(b2_sell_profit))
print("Sum profit b2 sell", Profit_sell)
Accuracy_sell = round((b2_sell["Profit"] > 0).sum() / len(b2_sell["Profit"]),2)
print(f"Accuracy {Accuracy_sell*100:.2f}%")
standard_sell = ((b2_sell["Profit"] > 0).sum()*TPSL_base)-((b2_sell["Profit"] < 0).sum()*TPSL_base)
error_sell = Profit_sell - standard_sell
print("Error sell",error_sell)

print()

print("------Overall MODEL v2-------")
Profit = Profit_buy + Profit_sell
print("Profit",Profit)
Accuracy = (Accuracy_buy + Accuracy_sell) / 2
print(f"Accuracy {Accuracy*100:.2f}%")
Error = error_sell + error_buy
print("Error",Error)

------MODEL v2--------
All Samples 30

Buy samples:  18
Sum profit b2 buy -7.17
Accuracy 50.00%
Error buy -7.17

Sell samples:  12
Sum profit b2 sell 12.52
Accuracy 67.00%
Error sell 0.0

------Overall MODEL v2-------
Profit 5.35
Accuracy 58.50%
Error -7.17


In [62]:
import pandas as pd

# create example dataframe
df = pd.DataFrame({
    'Time': ['2039-03-30', '2039-03-31', '2039-04-01'],
    'Price': [100, 101, 102]
})

# convert the 'Time' column to datetime format
df['Time'] = pd.to_datetime(df['Time'])

# filter by time > 2039-03-31
filtered_df = df[df['Time'] > '2039-03-31']

print(filtered_df)


        Time  Price
2 2039-04-01    102
